In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# LSTM 模型

     数据结构说明：
     
     1、基本数据
        1） 
        self: Open, Close, ..., Volume , INCOME, Flag   -> 为防止某些数据过大， 统一下做log，后面其它的数据统一采用该种方式。
        lstm data : line 1: [[open,....],[open,...]] 连续N个数据，通过LSTM 连续观察特征值，再进行分类
        
     2、网结构：
        
        1）
        Seq: 
            B, N , (O,H,L,..)  
            lstm --> B, N , 256  -->  liner --> B, 128  --> B,  64 --> B, 3
    

# 准备数据

In [2]:
from stock.common.sdata import StockData
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
code = '600016'
sd = StockData(code)
data = sd.combine_income(ndays=50)
data.loc[:,['Open','Close','High','Low','Volume','Amount']] =  \
    data.loc[:,['Open','Close','High','Low','Volume','Amount']].apply(lambda x: (x-np.mean(x))/np.std(x))

data_in , target = sd.package_data(data=data)
data_in = [x.values for x in data_in]
# data_a.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)),axis=1)
# for item in data_in:
#     # 归一化处理 , 最大值与最小值非常容易受异常点影响，所以这种方法鲁棒性较差，只适合传统精确小数据场景。
#     item['Volume'],item['Amount'] = np.log(item['Volume']),np.log(item['Amount'])
#     item = item.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)),axis=0)   
    # 标准化处理， 标准化在此应该更适合, 在已有样本足够多的情况下比较稳定，适合现代嘈杂大数据场景。
                                                                       

dlen = int(len(data_in) * 0.9)
train_data, train_target = torch.from_numpy(np.array(data_in[0:dlen])).float(), torch.from_numpy(np.array(target[0:dlen])).long()
test_data, test_target = torch.from_numpy(np.array(data_in[dlen:])).float(), torch.from_numpy(np.array(target[dlen:]).squeeze()).long()


# 网络模型 一

In [3]:
# 模型
import torch.nn  as nn
class Net(nn.Module):
    def __init__(self, n_feature, n_hidden=256, n_target=3):
        super(Net,self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature,hidden_size=n_hidden,batch_first=True)
        self.layer = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, n_target),
            nn.ReLU()
        )
    
    def forward(self, data):
        x, _ = self.lstm(data)
        x = self.layer(x)
        x = x[:,-1,:]
        return x

In [19]:
print(test_data.size())
out = net(test_data)
pred = torch.max(F.softmax(out), 1)[1]
print(pred)

torch.Size([286, 5, 6])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 训练网络
import torch.optim as optim
import torch.nn.functional as F
import os
import copy
import torch.nn.init as init
# from tensorboardX import SummaryWriter
from stock.common.log import Logger


# 网络参数初始化
def weight_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
#         init.xavier_normal_(m.bias.data)
    elif isinstance(m, nn.LSTM):
        init.xavier_normal_(m.all_weights[0][0])
        init.xavier_normal_(m.all_weights[0][1])        

#   tensorboard --logdir /home/hecong/dplearn/base/log

# writer = SummaryWriter("/home/hecong/dplearn/base/log")
logger = Logger('/home/hecong/dplearn/base/log')

feature = train_data.shape[2]
dataset = TensorDataset(train_data, train_target)
train_loader = DataLoader(dataset=dataset, batch_size=16, shuffle=True)

net = Net(n_feature=feature)
net.apply(weight_init)


if os.path.exists('net_model.pkl'):
    print('加载上次训练模型...')
    net.load_state_dict(torch.load('net_model.pkl'))
    

loss_function = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=1e-3)
optimizer = optim.Adam(net.parameters(), lr=0.01)


epcho = 10000
for epcho in range(epcho):
    run_loss = 0
    for step, (x,y) in enumerate(train_loader):
        net.train()
        optimizer.zero_grad()
        target = y.view(-1)
        out = net(x)
#         print(out)
#         pred = torch.max(F.softmax(out), 1)[1]
#         print(pred)
#         print(target)
        loss = loss_function(out,target)
        run_loss += loss
#         loss.register_hook(lambda g: print(g))
        loss.backward()
        optimizer.step()
    if epcho % 5 == 0:
        
        print('epcho {} step {} loss {:.4f}'.format(epcho,step, run_loss/step))
#         print()
        for name, param in net.layer.named_parameters():
            logger.histo_summary('layer/param/' + name, param.clone().cpu().data.numpy(), epcho)
            if param.grad is not None:
                logger.histo_summary('layer/grad/' + name, param.grad.clone().cpu().data.numpy(), epcho)
            
        for name, param in net.lstm.named_parameters():
            logger.histo_summary('lstm/param/' + name, param.clone().cpu().data.numpy(), epcho)
            if param.grad is not None:
                logger.histo_summary('lstm/grad/' + name, param.grad.clone().cpu().data.numpy(), epcho)

        net.eval()
        validate_out = net(test_data)
        pred = torch.max(F.softmax(validate_out), 1)[1]
        correct = sum(pred.data.numpy() == test_target.data.numpy())
        print('correct number : %s (%s)' % (correct, (correct / len(test_target.data.numpy()))))
        _wts = copy.deepcopy(net.state_dict())  
        torch.save(_wts,'net_model.pkl')       
        logger.scalar_summary('loss', (run_loss/step), epcho)
        logger.scalar_summary('correct', (correct / len(test_target.data.numpy())), epcho)
        
        
        
    

epcho 0 step 157 loss 0.5783


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:76: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


correct number : 193 (0.6868327402135231)
epcho 5 step 157 loss 0.5441
correct number : 193 (0.6868327402135231)
epcho 10 step 157 loss 0.5376
correct number : 193 (0.6868327402135231)
epcho 15 step 157 loss 0.5274
correct number : 193 (0.6868327402135231)
epcho 20 step 157 loss 0.5075
correct number : 193 (0.6868327402135231)
epcho 25 step 157 loss 0.5045
correct number : 193 (0.6868327402135231)
epcho 30 step 157 loss 0.4823
correct number : 193 (0.6868327402135231)
epcho 35 step 157 loss 0.4514
correct number : 125 (0.44483985765124556)
epcho 40 step 157 loss 0.4461
correct number : 128 (0.4555160142348754)
epcho 45 step 157 loss 0.4172
correct number : 142 (0.505338078291815)
epcho 50 step 157 loss 0.3966
correct number : 138 (0.49110320284697506)
epcho 55 step 157 loss 0.3925
correct number : 145 (0.5160142348754448)
epcho 60 step 157 loss 0.3900
correct number : 140 (0.498220640569395)
epcho 65 step 157 loss 0.3692
correct number : 130 (0.4626334519572954)
epcho 70 step 157 loss 

epcho 565 step 157 loss 0.1814
correct number : 140 (0.498220640569395)
epcho 570 step 157 loss 0.1217
correct number : 149 (0.5302491103202847)
epcho 575 step 157 loss 0.1982
correct number : 129 (0.45907473309608543)
epcho 580 step 157 loss 0.2424
correct number : 162 (0.5765124555160143)
epcho 585 step 157 loss 0.1629
correct number : 119 (0.4234875444839858)
epcho 590 step 157 loss 0.1140
correct number : 138 (0.49110320284697506)
epcho 595 step 157 loss 0.1168
correct number : 130 (0.4626334519572954)
epcho 600 step 157 loss 0.1847
correct number : 124 (0.4412811387900356)
epcho 605 step 157 loss 0.1444
correct number : 113 (0.40213523131672596)
epcho 610 step 157 loss 0.1765
correct number : 117 (0.41637010676156583)
epcho 615 step 157 loss 0.1685
correct number : 123 (0.4377224199288256)
epcho 620 step 157 loss 0.1932
correct number : 147 (0.5231316725978647)
epcho 625 step 157 loss 0.1701
correct number : 123 (0.4377224199288256)
epcho 630 step 157 loss 0.1524
correct number : 

In [10]:
# dd = iter(train_loader)
# x, y = next(dd)
# out = net(x)
# print(out.size())

print(train_data.size())


torch.Size([2566, 5, 6])
